Я решила обучить нейронную сеть распознавать написанные от руки числа. 

Сначала нужно подключить GPU, для этого выбираем Runtime ->  Change runtime type -> GPU


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [3]:
!nvidia-smi

Mon Jul 26 09:20:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    31W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [6]:

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.2))])

# Скачивание обучающего множества изображений из набора данных MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Создаем генератор батчей для обучающей выборки
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# Скачивание тестового множества изображений из набора данных MNIST
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Создаем генератор батчей для тестовой выборки
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3', '4', '5', '6',
          '7', '8', '9')

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # Конструктор нашей сети, в котором описываются все слои, которые будут в ней использоваться
    def __init__(self):
        super(Net, self).__init__()
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(1, 4, 3)
        self.conv2 = nn.Conv2d(4, 8, 3)
        self.conv3 = nn.Conv2d(8, 16, 3)
        
        self.fc1 = nn.Linear(16 * 3 * 3, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 10)
        
    # Опишем последовательность работы слоев сети
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x)) 
        x = x.view(-1, 16 * 3 * 3)
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x)) 
        x = self.fc4(x)
        return x
    
# Создаем экземпляр нашей сети   
net = Net()
# Перенесем сеть на указанное устройство
net = net.to(device)

In [8]:
N_EPOCHS = 10          # Количество эпох обучения
L_RATE = 0.001         # Скорость обучения
MOMENTUM = 0.8         # Момент
PRINT_EVERY = 500      # интервал для вывода результатов

In [9]:
import torch.optim as optim

# Определим функцию ошибки
criterion = nn.CrossEntropyLoss()
# Создадим оптимизатор
optimizer = optim.SGD(net.parameters(), lr=L_RATE, momentum=MOMENTUM)

In [10]:
def compute_acc(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct/ total

In [11]:
# Перебираем эпохи
for epoch in range(1, N_EPOCHS + 1):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % PRINT_EVERY == 0:
            print('EPOCH: {}, iter: {} | loss: {:.4f}'.format(epoch, i, running_loss / PRINT_EVERY))
            running_loss = 0.0
            
    epoch_acc = compute_acc(net)
    print('Accuracy of the network: {:.4}%'.format(epoch_acc))
    
    torch.save(net.state_dict(), './trained_net_{}_{:.4f}.weight'.format(epoch, epoch_acc))
print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


EPOCH: 1, iter: 0 | loss: 0.0046
EPOCH: 1, iter: 500 | loss: 2.3068
Accuracy of the network: 10.36%
EPOCH: 2, iter: 0 | loss: 0.0046
EPOCH: 2, iter: 500 | loss: 2.2996
Accuracy of the network: 11.93%
EPOCH: 3, iter: 0 | loss: 0.0046
EPOCH: 3, iter: 500 | loss: 2.2916
Accuracy of the network: 24.4%
EPOCH: 4, iter: 0 | loss: 0.0045
EPOCH: 4, iter: 500 | loss: 2.2453
Accuracy of the network: 45.65%
EPOCH: 5, iter: 0 | loss: 0.0030
EPOCH: 5, iter: 500 | loss: 1.1402
Accuracy of the network: 84.55%
EPOCH: 6, iter: 0 | loss: 0.0014
EPOCH: 6, iter: 500 | loss: 0.4674
Accuracy of the network: 89.8%
EPOCH: 7, iter: 0 | loss: 0.0006
EPOCH: 7, iter: 500 | loss: 0.3294
Accuracy of the network: 92.4%
EPOCH: 8, iter: 0 | loss: 0.0002
EPOCH: 8, iter: 500 | loss: 0.2381
Accuracy of the network: 94.18%
EPOCH: 9, iter: 0 | loss: 0.0003
EPOCH: 9, iter: 500 | loss: 0.1906
Accuracy of the network: 95.12%
EPOCH: 10, iter: 0 | loss: 0.0005
EPOCH: 10, iter: 500 | loss: 0.1604
Accuracy of the network: 95.9%
Fi

Таким образом, получилось, что наша нейронная сеть обучилась хорошо и показывает точность вычислений = 95.9%